# Aufgabe:
- Zweitquantisierung
- Fermin-zu-Qubit-Mapping (z.B. Jordan-Wigner)
- Validierung der Hamiltonians
Deliverable: Dokumentiertes Mapping + Tests

In [14]:
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyMinimumEigensolver
from scipy.linalg import eigh

In [15]:
geometry="H 0.0 0.0 0.0; H 0.0 0.0 0.74"

In [16]:
driver=PySCFDriver(
    atom=geometry,
    basis="sto-3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [17]:
problem=driver.run()

# Zweitquantisierung

In [18]:
fermionic_hamiltonian=problem.hamiltonian.second_q_op()
print(fermionic_hamiltonian)

Fermionic Operator
number spin orbitals=4, number terms=36
  0.3373779634072241 * ( +_0 +_0 -_0 -_0 )
+ 0.33185570067540676 * ( +_0 +_1 -_1 -_0 )
+ 0.3373779634072241 * ( +_0 +_2 -_2 -_0 )
+ 0.33185570067540676 * ( +_0 +_3 -_3 -_0 )
+ 0.09060523100759851 * ( +_0 +_0 -_1 -_1 )
+ 0.09060523100759851 * ( +_0 +_1 -_0 -_1 )
+ 0.09060523100759851 * ( +_0 +_2 -_3 -_1 )
+ 0.09060523100759851 * ( +_0 +_3 -_2 -_1 )
+ 0.09060523100759851 * ( +_1 +_0 -_1 -_0 )
+ 0.09060523100759851 * ( +_1 +_1 -_0 -_0 )
+ 0.09060523100759851 * ( +_1 +_2 -_3 -_0 )
+ 0.09060523100759851 * ( +_1 +_3 -_2 -_0 )
+ 0.33185570067540676 * ( +_1 +_0 -_0 -_1 )
+ 0.3488257522452315 * ( +_1 +_1 -_1 -_1 )
+ 0.33185570067540676 * ( +_1 +_2 -_2 -_1 )
+ 0.3488257522452315 * ( +_1 +_3 -_3 -_1 )
+ 0.3373779634072241 * ( +_2 +_0 -_0 -_2 )
+ 0.33185570067540676 * ( +_2 +_1 -_1 -_2 )
+ 0.3373779634072241 * ( +_2 +_2 -_2 -_2 )
+ 0.33185570067540676 * ( +_2 +_3 -_3 -_2 )
+ 0.09060523100759851 * ( +_2 +_0 -_1 -_3 )
+ 0.09060523100759851 *

# Mapping

In [19]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(fermionic_hamiltonian)
print(qubit_hamiltonian)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.12062523+0.j,
  0.17141283+0.j,  0.16868898+0.j, -0.22343154+0.j,  0.16592785+0.j,
  0.04530262+0.j,  0.04530262+0.j,  0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])


# Validierung der Hamiltonians

In [20]:
matrix = qubit_hamiltonian.to_matrix()

In [21]:
is_hermitian = np.allclose(matrix, matrix.conj().T)
print(f"Matrix ist hermitesch: {is_hermitian}")

Matrix ist hermitesch: True


In [22]:
solver = NumPyMinimumEigensolver()
result = solver.compute_minimum_eigenvalue(qubit_hamiltonian)
print(f"Referenz-Grundzustandsenergie: {result.eigenvalue.real:.6f} H")

Referenz-Grundzustandsenergie: -1.852388 H


In [23]:
eigenvalues = np.linalg.eigvalsh(matrix)
ground_state_energy = min(eigenvalues).real
print(f"Grundzustandsenergie: {ground_state_energy:.6f} H")

Grundzustandsenergie: -1.852388 H
